# Métodos de seleção de variáveis abordados nesse capítulo:
1. Recursive Feature Elimination
2. Ensemble de métodos de seleção de variáveis
3. Recursive Feature Selection

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import RFE, SequentialFeatureSelector
import warnings
warnings.filterwarnings('ignore')

In [2]:
ansur_df = pd.read_csv("../dados/ansur_df.csv")

In [3]:
#ansur_df = ansur_df[["Gender", "chestdepth", "handlength", "neckcircumference", "shoulderlength", "earlength"]]
ansur_df=ansur_df.select_dtypes(include=["int"])

In [4]:
ansur_df.head()

,abdominalextensiondepthsitting,acromialheight,acromionradialelength,anklecircumference,axillaheight,balloffootcircumference,balloffootlength,biacromialbreadth,bicepscircumferenceflexed,bicristalbreadth,...,trochanterionheight,verticaltrunkcircumferenceusa,waistbacklength,waistbreadth,waistcircumference,waistdepth,waistfrontlengthsitting,waistheightomphalion,wristcircumference,wristheight
0,266,1467,337,222,1347,253,202,401,369,274,...,919,1700,501,329,933,240,440,1054,175,853
1,233,1395,326,220,1293,245,193,394,338,257,...,918,1627,432,316,870,225,371,1054,167,815
2,287,1430,341,230,1327,256,196,427,408,261,...,918,1678,472,329,964,255,411,1041,180,831
3,234,1347,310,230,1239,262,199,401,359,262,...,847,1625,461,315,857,205,399,968,176,793
4,250,1585,372,247,1478,267,224,435,356,263,...,1090,1679,467,303,868,214,379,1245,188,954


In [5]:
ansur_df.shape

(4082, 91)

In [6]:
ansur_df.loc[:2000, "Gender"]="Female"

In [7]:
X=ansur_df.drop(["Gender"], axis=1)
y=ansur_df["Gender"]

In [8]:
y=y.fillna("Male")

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [10]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

## O método do scikit learn chamado Recursive Feature Elimination(RFE) descarta variáveis baseado em alguns modelos específicos que possuem como atributos coef_ ou feature_importances_. Servem tanto para modelos de regressão ou classificação. Para usar com validação cruzada, existe a função RFECV. Esses modelos geralmente são modelos lineares ou modelos baseados em árvores. Alguns deles são:
1. Regressão Linear
2. Regressão Lasso
3. Regressão Ridge
4. Floresta Aleatória
5. Árvore de Decisão
6. Gradient Tree Boosting
7. Regressão Logística

### Nesse exemplo, vou usar uma Regressão logística e o RFE já usa automaticamente o atributo coef_ para descartar ou manter as variáveis. Nesse é muito importante que as variáveis estejam na mesma escala para podermos compararmos os valores dos coeficientes.

In [11]:
lr = RFE(estimator=LogisticRegression(), n_features_to_select=5, step=5)
lr.fit(X_train_std, y_train)

RFE(estimator=LogisticRegression(), n_features_to_select=5, step=5)

In [12]:
mask_lr=lr.support_

In [13]:
X_reduced = X.loc[:, mask_lr]
print(X_reduced.columns)

Index(['buttockkneelength', 'buttockpopliteallength', 'kneeheightmidpatella',
       'lateralfemoralepicondyleheight', 'tibialheight'],
      dtype='object')


### Nesse exemplo, vou usar uma Floresta Aleatória e o RFE já usa automaticamente o atributo feature_importances_ para descartar ou manter as variáveis. Nesse é muito importante que as variáveis estejam na mesma escala para podermos compararmos os valores dos coeficientes.

In [14]:
rf = RFE(estimator=RandomForestClassifier(),n_features_to_select=6, step=5)
rf.fit(X_train,y_train)

RFE(estimator=RandomForestClassifier(), n_features_to_select=6, step=5)

In [15]:
mask_rf=rf.support_
print(mask_rf)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True False False  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False  True False False False False False False False False
 False False False  True False False False  True False False False False
 False False False  True False False False]


In [16]:
X_reduced = X_train.loc[:, mask_rf]
print(X_reduced.columns)

Index(['chestheight', 'crotchlengthposterioromphalion',
       'overheadfingertipreachsitting', 'tenthribheight', 'tibialheight',
       'waistfrontlengthsitting'],
      dtype='object')


### Nesse exemplo, irei usar um GradientBoosting:

In [17]:
gb = RFE(estimator=GradientBoostingClassifier(),n_features_to_select=6, step=5)
gb.fit(X_train,y_train)

RFE(estimator=GradientBoostingClassifier(), n_features_to_select=6, step=5)

In [18]:
mask_gb=gb.support_
print(mask_gb)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True False False  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False  True False False False False False False False False
 False False False  True False False False  True False False False False
 False False False  True False False False]


In [19]:
X_reduced = X_train.loc[:, mask_gb]
print(X_reduced.columns)

Index(['chestheight', 'crotchlengthposterioromphalion',
       'overheadfingertipreachsitting', 'tenthribheight', 'tibialheight',
       'waistfrontlengthsitting'],
      dtype='object')


## Podemos usar o resultados de vários modelos para selecionar variáveis. Nesse caso, vou descartar variáveis quando pelo menos 2 dos 3 modelos utilizados também descartarem as mesmas variáveis.

In [20]:
votes = np.sum([mask_lr, mask_rf, mask_gb], axis=0)
print(votes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 2 0 0 2 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0
 0 2 0 0 0 3 0 0 0 0 0 0 0 2 0 0 0]


In [21]:
mask = votes >= 2

In [22]:
reduced_X = X_train.loc[:, mask]
print(X_reduced.columns)

Index(['chestheight', 'crotchlengthposterioromphalion',
       'overheadfingertipreachsitting', 'tenthribheight', 'tibialheight',
       'waistfrontlengthsitting'],
      dtype='object')


# O método Recursive Feature Selector elimina ou adiciona variáveis recursivamente de acordo com alguma métrica de performance selecionada. É sempre feita uma validação cruzada. 
### Como esse método só adiciona ou remove de uma por uma, dependendo do número do variáveis pode demorar bastante e outros métodos são preferíveis.

In [24]:
X_train=X_train.iloc[:, :15]

In [25]:
sfs = SequentialFeatureSelector(RandomForestClassifier(), n_features_to_select=5, direction="forward")
sfs.fit(X_train,y_train)

SequentialFeatureSelector(estimator=RandomForestClassifier(),
                          n_features_to_select=5)

In [26]:
mask = sfs.get_support()

In [28]:
reduced_X = X_train.loc[:, mask]
print(X_reduced.columns)

Index(['chestheight', 'crotchlengthposterioromphalion',
       'overheadfingertipreachsitting', 'tenthribheight', 'tibialheight',
       'waistfrontlengthsitting'],
      dtype='object')
